In [1]:
# Centralized sonar defaults (inserted by sweep)
from utils.sonar_config import SONAR_VIS_DEFAULTS
sonar_config = SONAR_VIS_DEFAULTS.copy()
# Backwards-compatible variable names used in older notebooks
RANGE_MIN_M = sonar_config['range_min_m']
RANGE_MAX_M = sonar_config['range_max_m']
DISPLAY_RANGE_MAX_M = sonar_config['display_range_max_m']

In [2]:
# 🔧 DISTANCE VALIDATION CONFIGURATION
# ===================================
# Configure the new distance validation filter

from utils.sonar_config import IMAGE_PROCESSING_CONFIG

print(f"🎛️  Current distance validation setting: {IMAGE_PROCESSING_CONFIG.get('use_distance_validation', True)}")

# Option 1: Keep distance validation ON (default - filters invalid measurements)
# IMAGE_PROCESSING_CONFIG['use_distance_validation'] = True

# Option 2: Turn distance validation OFF (raw distances, may have jumps/errors)  
# IMAGE_PROCESSING_CONFIG['use_distance_validation'] = False

print("\n📋 Distance Validation Options:")
print("   ✅ use_distance_validation = True  → Filters negative/out-of-bounds distances")
print("   ⚠️  use_distance_validation = False → Raw distances (may have jumps/errors)")
print(f"\n🎯 Filter Description:")
print("   • Ensures distance is never negative")
print("   • Ensures distance is within image bounds (0 to image height)")
print("   • Uses last valid distance when current measurement is invalid")
print(f"   • Current setting: {'ENABLED' if IMAGE_PROCESSING_CONFIG.get('use_distance_validation', True) else 'DISABLED'}")

🎛️  Current distance validation setting: True

📋 Distance Validation Options:
   ✅ use_distance_validation = True  → Filters negative/out-of-bounds distances
   ⚠️  use_distance_validation = False → Raw distances (may have jumps/errors)

🎯 Filter Description:
   • Ensures distance is never negative
   • Ensures distance is within image bounds (0 to image height)
   • Uses last valid distance when current measurement is invalid
   • Current setting: ENABLED


In [3]:
# 🎯 ELLIPSE TRACKING SMOOTHNESS CONFIGURATION
# ============================================
# Configure ellipse movement smoothing to prevent jumping/flickering

from utils.sonar_config import TRACKING_CONFIG

print("🎛️  ELLIPSE SMOOTHING PARAMETERS:")
print(f"   ellipse_smoothing_alpha: {TRACKING_CONFIG.get('ellipse_smoothing_alpha', 0.2)}")
print(f"   ellipse_max_movement_pixels: {TRACKING_CONFIG.get('ellipse_max_movement_pixels', 4.0)}")

print("\n🔧 TUNING GUIDELINES:")
print("   ellipse_smoothing_alpha (0.0-1.0):")
print("      ↓ Lower (0.1) = Very smooth, slow to adapt to movement")  
print("      ↑ Higher (0.5) = More responsive, but may be jittery")
print("   ellipse_max_movement_pixels:")
print("      ↓ Lower (2.0) = Prevents jumping, very stable tracking")
print("      ↑ Higher (8.0) = Allows faster movement, may jump to noise")

# OPTIONAL: Adjust parameters for smoother tracking
# TRACKING_CONFIG['ellipse_smoothing_alpha'] = 0.15  # Smoother movement
# TRACKING_CONFIG['ellipse_max_movement_pixels'] = 3.0  # Limit jump distance

print(f"\n🎯 Current behavior:")
print(f"   Smoothing: {'Low (very smooth)' if TRACKING_CONFIG.get('ellipse_smoothing_alpha', 0.2) < 0.2 else 'Medium' if TRACKING_CONFIG.get('ellipse_smoothing_alpha', 0.2) < 0.4 else 'High (responsive)'}")
print(f"   Movement limit: {TRACKING_CONFIG.get('ellipse_max_movement_pixels', 4.0)} pixels per frame")

🎛️  ELLIPSE SMOOTHING PARAMETERS:
   ellipse_smoothing_alpha: 0.3
   ellipse_max_movement_pixels: 8.0

🔧 TUNING GUIDELINES:
   ellipse_smoothing_alpha (0.0-1.0):
      ↓ Lower (0.1) = Very smooth, slow to adapt to movement
      ↑ Higher (0.5) = More responsive, but may be jittery
   ellipse_max_movement_pixels:
      ↓ Lower (2.0) = Prevents jumping, very stable tracking
      ↑ Higher (8.0) = Allows faster movement, may jump to noise

🎯 Current behavior:
   Smoothing: Medium
   Movement limit: 8.0 pixels per frame


# Simple Image Analysis with CV2

This notebook demonstrates:
1. **Pick a frame** from NPZ files and save it locally
2. **Use standard cv2 functions** directly for image processing
3. **Experiment** with different OpenCV operations

In [4]:
from pathlib import Path
import utils.sonar_image_analysis as iau


# Prefer selecting by bag ID (part of the NPZ filename) instead of a numeric index.
# Set TARGET_BAG to a substring that appears in the NPZ filename (e.g. '2024-08-22_14-29-05')
# '2024-08-22_14-47-39'
# '2024-08-22_14-29-05'
# '2024-08-20_14-31-29' HARD
TARGET_BAG = '2024-08-20_14-31-29'  # change this to your desired bag ID
from utils.sonar_config import EXPORTS_DIR_DEFAULT, EXPORTS_SUBDIRS
EXPORTS_FOLDER = Path(EXPORTS_DIR_DEFAULT)

# If you keep your raw .bag files or raw data on an external drive, point DATA_DIR there.
# Example (your external disk): /Volumes/LaCie/SOLAQUA/raw_data
DATA_DIR = Path("/Volumes/LaCie/SOLAQUA/raw_data")
print(f"Using DATA_DIR = {DATA_DIR}")

# Find NPZ files and pick the one matching TARGET_BAG
files = iau.get_available_npz_files()
if not files:
    raise FileNotFoundError(f"No NPZ files found in configured exports outputs (looked under {EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('outputs','outputs')})")
matches = [p for p in files if TARGET_BAG in p.name]
if not matches:
    # Help the user by listing available NPZ files
    print(f'No NPZ file matched TARGET_BAG={TARGET_BAG!r}')
    print('Available NPZ files:')
    for i,p in enumerate(files):
        print(f'  {i}: {p.name}')
    raise ValueError(f'No NPZ file contains "{TARGET_BAG}" in its name')

# If multiple matches, choose the most recently modified one
selected = max(matches, key=lambda p: p.stat().st_mtime)
NPZ_FILE_INDEX = files.index(selected)
print(f'Selected NPZ file: {selected.name} (index={NPZ_FILE_INDEX})')

Using DATA_DIR = /Volumes/LaCie/SOLAQUA/raw_data
Selected NPZ file: 2024-08-20_14-31-29_data_cones.npz (index=8)


## Distance Analysis Over Time

Now let's perform a comprehensive analysis of the red line distance over time. The red line represents the major axis of the detected elongated contour (likely a fishing net), and we'll track how this distance changes throughout the video sequence.

In [5]:
# Now using the NEW unified SonarDataProcessor with elliptical AOI support!
# This gives us much better net tracking compared to the old rectangular AOI

# First reload the module to get the latest elliptical AOI changes
import importlib
importlib.reload(iau)

# Create the unified processor
processor = iau.SonarDataProcessor()

# Use the new Distance Analysis Engine with elliptical AOI
engine = iau.DistanceAnalysisEngine()
distance_results = engine.analyze_npz_sequence(
    npz_file_index=NPZ_FILE_INDEX,    
    frame_start=1,        # Start from frame 1
    frame_count=1500,      # Analyze all frames from the video
    frame_step=1          # Every frame
)

print(f"✅ Analyzed {len(distance_results)} frames using elliptical AOI")
print(f"🎯 Detection success rate: {distance_results['detection_success'].mean():.1%}")
print(f"📊 Elliptical tracking active: {(distance_results['tracking_status'].str.contains('ELLIPSE')).sum()} frames")

=== DISTANCE ANALYSIS FROM NPZ ===
Analyzing: /Volumes/LaCie/SOLAQUA/exports/outputs/2024-08-20_14-31-29_data_cones.npz
Processing 1059 frames from 1 (step=1)
Processing 1059 frames from 1 (step=1)
  Processed 50/1059 frames (Success rate: 100.0%)
  Processed 50/1059 frames (Success rate: 100.0%)
  Processed 100/1059 frames (Success rate: 100.0%)
  Processed 100/1059 frames (Success rate: 100.0%)
  Processed 150/1059 frames (Success rate: 100.0%)
  Processed 150/1059 frames (Success rate: 100.0%)
  Processed 200/1059 frames (Success rate: 100.0%)
  Processed 200/1059 frames (Success rate: 100.0%)
  Processed 250/1059 frames (Success rate: 100.0%)
  Processed 250/1059 frames (Success rate: 100.0%)
  Processed 300/1059 frames (Success rate: 100.0%)
  Processed 300/1059 frames (Success rate: 100.0%)
  Processed 350/1059 frames (Success rate: 100.0%)
  Processed 350/1059 frames (Success rate: 100.0%)
  Processed 400/1059 frames (Success rate: 100.0%)
  Processed 400/1059 frames (Success ra

In [6]:
# Create video using the CORE simplified processor with ELLIPTICAL AOI
# This will now show the yellow ELLIPTICAL AOI instead of rectangular!

# Use the enhanced video creation with CORE processor
video_path = iau.create_enhanced_contour_detection_video_with_processor(
    npz_file_index=NPZ_FILE_INDEX,          # Which NPZ file to use
    frame_start=1,           # Starting frame
    frame_count=1500,         # Start with 1500 frames for testing
    frame_step=1,            # Step between frames
    output_path=Path(EXPORTS_DIR_DEFAULT) / EXPORTS_SUBDIRS.get('videos','videos') / 'elliptical_aoi_tracking.mp4',
    processor=processor      # Use our CORE processor with ellipse tracking
)

print(f"✅ Video created with ELLIPTICAL AOI visualization!")
print(f"🎯 The AOI should now appear as a yellow ELLIPSE that smoothly tracks the net")
print(f"📁 Video saved to: {video_path}")

=== ENHANCED ELLIPTICAL AOI VIDEO CREATION ===
Creating video with elliptical AOI tracking...
Frames: 1500, step: 1
✅ Processing 1059 frames with elliptical AOI...
✅ Processing 1059 frames with elliptical AOI...
Processed 10/1059 frames | Ellipse tracking: 0
Processed 10/1059 frames | Ellipse tracking: 0
Processed 20/1059 frames | Ellipse tracking: 0
Processed 20/1059 frames | Ellipse tracking: 0
Processed 30/1059 frames | Ellipse tracking: 0
Processed 30/1059 frames | Ellipse tracking: 0
Processed 40/1059 frames | Ellipse tracking: 0
Processed 40/1059 frames | Ellipse tracking: 0
Processed 50/1059 frames | Ellipse tracking: 0
Processed 50/1059 frames | Ellipse tracking: 0
Processed 60/1059 frames | Ellipse tracking: 0
Processed 60/1059 frames | Ellipse tracking: 0
Processed 70/1059 frames | Ellipse tracking: 0
Processed 70/1059 frames | Ellipse tracking: 0
Processed 80/1059 frames | Ellipse tracking: 0
Processed 80/1059 frames | Ellipse tracking: 0
Processed 90/1059 frames | Ellipse t

## 🎯 Elliptical AOI Visualization

The AOI (Area of Interest) has been updated to use **elliptical tracking** instead of rectangular:

### What you'll see in the video:
- **Yellow Ellipse**: The elliptical AOI that tracks the net contour
- **Magenta Ellipse**: The fitted ellipse around the best detected contour  
- **Red Line**: The major axis of the detected net (90° rotated)
- **Green Contour**: The best detected net contour
- **Blue Dot**: Intersection with center beam for distance measurement

### Key Improvements:
1. **Smooth Movement**: Ellipse uses exponential smoothing to prevent jumping
2. **Better Shape Matching**: Elliptical AOI better matches elongated net shape
3. **More Restrictive**: Elliptical area excludes more non-net pixels than rectangular
4. **Adaptive**: AOI adapts to the net's actual shape and orientation

## Convert to Real-World Distances

Now let's convert the pixel distances to real-world distances using the fact that the entire sonar image represents a 10x10 meter area.

In [7]:
# Auto-detect pixel->meter mapping from the selected NPZ (if available)
try:
    cones, ts, extent, meta = iau.load_cone_run_npz(selected)
    T, H, W = cones.shape
    x_min, x_max, y_min, y_max = extent
    width_m = float(x_max - x_min)
    height_m = float(y_max - y_min)
    px2m_x = width_m / float(W)
    px2m_y = height_m / float(H)
    pixels_to_meters_avg = 0.5 * (px2m_x + px2m_y)
    image_shape = (H, W)
    sonar_coverage_meters = max(width_m, height_m)
    print(f"Detected NPZ extent: x=[{x_min:.3f},{x_max:.3f}] m, y=[{y_min:.3f},{y_max:.3f}] m")
    print(f"Image shape from NPZ: H={H}, W={W}")
    print(f"meters/pixel: x={px2m_x:.6f}, y={px2m_y:.6f}, avg={pixels_to_meters_avg:.6f}")
except Exception as e:
    print("Could not read NPZ metadata:", e)
    print("Falling back to defaults from sonar_config.")
    from utils.sonar_config import CONE_H_DEFAULT, CONE_W_DEFAULT, DISPLAY_RANGE_MAX_M_DEFAULT
    image_shape = (CONE_H_DEFAULT, CONE_W_DEFAULT)
    sonar_coverage_meters = DISPLAY_RANGE_MAX_M_DEFAULT * 2  # approximate
    pixels_to_meters_avg = sonar_coverage_meters / max(image_shape)
    
# Use the computed pixels_to_meters_avg in downstream analysis
print(f"Using pixels_to_meters_avg = {pixels_to_meters_avg:.6f} m/px")

Detected NPZ extent: x=[-8.660,8.660] m, y=[0.000,10.000] m
Image shape from NPZ: H=700, W=900
meters/pixel: x=0.019245, y=0.014286, avg=0.016765
Using pixels_to_meters_avg = 0.016765 m/px


In [8]:
# Real-world distance analysis and plotting using utility function
iau.plot_real_world_distance_analysis(distance_results, image_shape=image_shape, sonar_coverage_meters=sonar_coverage_meters)

In [9]:
# 🔄 COMPARISON: SONAR vs DVL DISTANCE MEASUREMENTS
# =================================================
import utils.net_distance_analysis as sda

print(f"🎯 LOADING DVL DATA FOR COMPARISON: {TARGET_BAG}")
print("=" * 60)

# IMPORTANT: Pass the by_bag folder, not just the exports root
# The function expects the folder containing the CSV files
from utils.sonar_config import EXPORTS_SUBDIRS
BY_BAG_FOLDER = EXPORTS_FOLDER / EXPORTS_SUBDIRS.get('by_bag', 'by_bag')

# Load all distance data for the target bag
raw_data, distance_measurements = sda.load_all_distance_data_for_bag(TARGET_BAG, BY_BAG_FOLDER)

# Display what we loaded
print(f"\n📊 RAW DATA LOADED:")
for key, data in raw_data.items():
    if data is not None:
        print(f"   ✅ {key}: {len(data)} records")
    else:
        print(f"   ❌ {key}: None")

print(f"\n📏 DISTANCE MEASUREMENTS LOADED:")
for key, info in distance_measurements.items():
    data_len = len(info['data'])
    print(f"   ✅ {key}: {data_len} records - {info['description']}")

🎯 LOADING DVL DATA FOR COMPARISON: 2024-08-20_14-31-29
🎯 LOADING ALL DISTANCE DATA FOR BAG: 2024-08-20_14-31-29
📡 1. Loading Navigation Data...
   ✅ Loaded 559 navigation records
📡 2. Loading Guidance Data...
   ✅ Loaded 549 guidance records with ['error_net_distance', 'desired_net_distance', 'r_net_distance_d']
📡 3. Loading DVL Altimeter...
   ❌ DVL altimeter file not found
📡 4. Loading USBL...
   ✅ Loaded 33 USBL records
📡 5. Loading DVL Position...
   ✅ Loaded 283 DVL position records
📡 6. Loading Navigation Position...
   ✅ Loaded 278 navigation position records
📡 7. Loading INS Z Position...
   ❌ INS file not found

📊 LOADING SUMMARY:
   🎯 Target bag: 2024-08-20_14-31-29
   📁 Raw data loaded: 2/2
   📏 Distance measurements: 4

📊 RAW DATA LOADED:
   ✅ navigation: 559 records
   ✅ guidance: 549 records

📏 DISTANCE MEASUREMENTS LOADED:
   ✅ USBL_3D: 33 records - 3D acoustic position
   ✅ USBL_Depth: 33 records - USBL depth measurement
   ✅ DVL_Position: 283 records - 3D DVL position


In [10]:
try:
    fig, comparison_stats = iau.interactive_distance_comparison(distance_results, raw_data, sonar_coverage_m=sonar_coverage_meters, sonar_image_size=image_shape[0])
    if comparison_stats and isinstance(comparison_stats, dict) and 'error' in comparison_stats:
        print('Comparison did not run:', comparison_stats['error'])
    else:
        print('\nComparison stats summary:')
        from pprint import pprint
        pprint(comparison_stats)
        # Display the interactive plot
        fig.show()
except Exception as e:
    print('Error running interactive comparison:', e)

/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:496: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/Users/eirikvarnes/code/SOLAQUA/utils/sonar_image_analysis.py:503: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.




📊 SONAR vs DVL COMPARISON STATISTICS:
Sonar mean distance: 2.043 m
DVL mean distance:   1.968 m
Scale ratio (Sonar/DVL): 1.038x
Sonar duration: 67.1s (1059 frames)
DVL duration:   67.0s (559 records)

Comparison stats summary:
{'dvl_duration_s': 67.007197142,
 'dvl_mean_m': 1.9676565289337435,
 'dvl_records': 559,
 'scale_ratio': 1.0384467682649405,
 'sonar_duration_s': 67.07053097672778,
 'sonar_frames': 1059,
 'sonar_mean_m': 2.0433065635266563}


In [ ]:
# 🎥 Generate Video with Both DVL and Sonar Analysis Overlays
# ========================================================
import utils.sonar_and_foto_generation as sg

# Set up paths
VIDEO_SEQ_DIR = None  # Set to path if you want camera footage included

print("🎬 GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS")
print("=" * 60)
print(f"🎯 Target Bag: {TARGET_BAG}")
print(f"📊 DVL Data: {len(raw_data.get('navigation', []))} records")
print(f"📏 Sonar Analysis: {len(distance_results)} frames")
print(f"🎥 Camera: {'enabled' if VIDEO_SEQ_DIR else 'disabled'}")

# Generate the video with both overlays
try:
    video_path = sg.export_optimized_sonar_video(
        TARGET_BAG=TARGET_BAG,
        EXPORTS_FOLDER=EXPORTS_FOLDER,
        START_IDX=1,
        END_IDX=1000,  # Adjust as needed
        STRIDE=1,
        VIDEO_SEQ_DIR=VIDEO_SEQ_DIR,
        INCLUDE_NET=True,  # Enable DVL net distance overlay
        SONAR_DISTANCE_RESULTS=distance_results,  # Enable sonar analysis overlay
        NET_DISTANCE_TOLERANCE=0.5,
        NET_PITCH_TOLERANCE=2.0,  # Increased from 0.3 to 2.0 seconds for better pitch sync
    )
    print(f"\n✅ Video generated successfully!")
    print(f"📁 Output: {video_path}")
except Exception as e:
    print(f"❌ Error generating video: {e}")

🎬 GENERATING VIDEO WITH DUAL NET DISTANCE OVERLAYS
🎯 Target Bag: 2024-08-20_14-31-29
📊 DVL Data: 559 records
📏 Sonar Analysis: 1059 frames
🎥 Camera: disabled
🛠️ OPTIMIZED SONAR VIDEO
🎯 Target Bag: 2024-08-20_14-31-29
   Cone Size: 900x700
   Range: 0.0-5.0m | FOV: 120.0°
   🎥 Camera: disabled
   🕸  Net-line: enabled (dist tol=0.5s, pitch tol=2.0s)
   📊 Sonar Analysis: enabled
   Loading sonar data: sensor_sonoptix_echo_image__2024-08-20_14-31-29_video.csv
   ✅ Loaded 1060 sonar frames in 42.01s
   ✅ Loaded 559 navigation records in 0.01s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   Frames: 1..999 (step 1) => 999
   Natural FPS: 15.7
   ✅ Loaded 1060 sonar frames in 42.01s
   ✅ Loaded 559 navigation records in 0.01s
      Available: ['NetDistance', 'NetPitch', 'timestamp']
   Frames: 1..999 (step 1) => 999
   Natural FPS: 15.7

🎉 DONE! Wrote 999 frames to /Volumes/LaCie/SOLAQUA/exports/videos/2024-08-20_14-31-29_optimized_sync_withsonar_20240820_143131_621807+0200.mp4 @ 